In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
#Make sure tf 2.x is installed
!pip install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 121kB/s 
     |████████████████████████████████| 3.8MB 44.1MB/s 
     |████████████████████████████████| 450kB 72.6MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.2 which is incompatible.


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [84]:
tf.__version__

'2.0.0'

**1.) Read the dataset**

In [85]:
bank = pd.read_csv("/content/Churn.csv")
bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0



**2.) Drop the columns which are unique for all users like IDs (2.5 points)**

In [86]:
bank.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
bank.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [87]:
bank.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
Exited,-0.027094,0.285323,-0.014001,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


**3.) Distinguish the feature and target set (2.5 points)**

In [0]:
# Copy the independent/feature variables into X dataframe. 
# Since 'Exited' is dependent variable drop it
X= np.array(bank.drop(columns=(['Exited'])))

# Copy the 'Exited' column alone into the y dataframe.
# This is the target set
Y = np.array(bank['Exited'])

In [91]:
X.shape

(10000, 10)

In [92]:
Y.shape

(10000,)

In [0]:
# Encoding categorical data# 
#Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Geography
labelencoder_X = LabelEncoder()
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
# Gender
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [94]:
Y

array([1, 0, 1, ..., 1, 1, 0])

In [95]:
# Giving ordinal feature to our variables
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


**4.) Divide the data set into train and test sets**

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

**5.) Normalize the train and test data (2.5 points)**

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


**6.) Initialize & build the model (10 points)**

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model = tf.keras.models.Sequential()

#Input Layer
model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11 ))
#model.add(tf.keras.layers.Dense(6, input_shape=(11,)))

#Adding a layer
model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

#Output layer
model.add(tf.keras.layers.Dense(units = 1, kernel_initializer = 'uniform', activation = 'softmax'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [107]:
#Summarise the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [108]:
#Train the model
model.fit(X_train,Y_train,
          validation_data=(X_test,Y_test),
          epochs=15,
          batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 1s 127us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 2/15
8000/8000 [==============================] - 0s 53us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 3/15
8000/8000 [==============================] - 0s 54us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 4/15
8000/8000 [==============================] - 0s 54us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 5/15
8000/8000 [==============================] - 0s 54us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 6/15
8000/8000 [==============================] - 0s 52us/sample - loss: 12.2053 - accuracy: 0.2040 - val_loss: 12.2283 - val_accuracy: 0.2025
Epoch 7/15
8000/8000 [==============================] - 0

In [109]:
Y_train.shape

(8000,)

In [110]:
Y_test.shape

(2000,)

**7.) Optimize the model (5 points)**

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model = tf.keras.models.Sequential()

#Input Layer
model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11 ))
#model.add(tf.keras.layers.Dense(6, input_shape=(11,)))

#Adding a layer
model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

#Output layer
model.add(tf.keras.layers.Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [112]:
#Train the model
model.fit(X_train,Y_train,
          validation_data=(X_test,Y_test),
          epochs=15,
          batch_size=30)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 1s 107us/sample - loss: 0.5804 - accuracy: 0.7955 - val_loss: 11138.1996 - val_accuracy: 0.7975
Epoch 2/15
8000/8000 [==============================] - 0s 55us/sample - loss: 0.4915 - accuracy: 0.7960 - val_loss: 12117.4109 - val_accuracy: 0.7975
Epoch 3/15
8000/8000 [==============================] - 0s 55us/sample - loss: 0.4888 - accuracy: 0.7960 - val_loss: 10494.5303 - val_accuracy: 0.7975
Epoch 4/15
8000/8000 [==============================] - 0s 55us/sample - loss: 0.4876 - accuracy: 0.7960 - val_loss: 11321.1326 - val_accuracy: 0.7975
Epoch 5/15
8000/8000 [==============================] - 0s 55us/sample - loss: 0.4867 - accuracy: 0.7960 - val_loss: 11024.9066 - val_accuracy: 0.7975
Epoch 6/15
8000/8000 [==============================] - 0s 53us/sample - loss: 0.4856 - accuracy: 0.7960 - val_loss: 10761.0262 - val_accuracy: 0.7975
Epoch 7/15
8000/8000 [=======================

**After optimizing the model we are getting an accuracy of 79%**

**8.) Predict the results using 0.5 as a threshold (5 points)**

In [113]:
#Predicting the Test set results
y_pred = model.predict(X_test)#Threshold of 50%
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

**9.) Print the Accuracy score and confusion matrix (2.5 points)**

In [115]:
#For the Confusion Matrix
from sklearn.metrics import confusion_matrix
cmt = confusion_matrix(Y_test, y_pred)
cmt

array([[1595,    0],
       [ 405,    0]])

In [116]:
#accuracy
print ((cmt[0][0] + cmt[1][1])/(cmt[0][0] +cmt[0][1] + cmt[1][0]+cmt[1][1]))

0.7975


In [118]:
#Accuracy
accuracy = (1595+0)/(2000)
accuracy

0.7975

**Conclusion :**
**Thus we are getting an accuracy of 79%**